## Deliverable 2. Create a Customer Travel Destinations Map.

In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [25]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,69.96,94,20,6.91,few clouds
1,1,Rikitea,PF,-23.1203,-134.9692,72.23,60,98,10.60,overcast clouds
2,2,Eyl,SO,7.9803,49.8164,83.23,67,61,24.07,broken clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,66.33,68,100,12.55,overcast clouds
4,4,Hermanus,ZA,-34.4187,19.2345,64.80,66,86,18.57,overcast clouds


In [26]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [27]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,72.23,60,98,10.60,overcast clouds
2,2,Eyl,SO,7.9803,49.8164,83.23,67,61,24.07,broken clouds
5,5,Victoria,HK,22.2855,114.1577,87.33,81,91,5.01,overcast clouds
8,8,Puerto Ayora,EC,-0.7393,-90.3518,76.96,94,65,4.00,broken clouds
12,12,Tual,ID,-5.6667,132.7500,81.55,78,50,22.86,scattered clouds
15,15,Benidorm,ES,38.5382,-0.1310,75.49,91,100,9.08,overcast clouds
22,22,Bandarbeyla,SO,9.4942,50.8122,80.29,76,50,25.19,scattered clouds
24,24,Kapaa,US,22.0752,-159.3190,73.72,80,40,9.04,scattered clouds
26,26,Hellin,ES,38.5106,-1.7010,74.98,45,100,11.61,overcast clouds
30,30,Palora,EC,-1.7000,-77.9333,71.71,73,86,2.37,overcast clouds


In [28]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [29]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how="any")
clean_df.count()

City_ID                299
City                   299
Country                299
Lat                    299
Lng                    299
Max Temp               299
Humidity               299
Cloudiness             299
Wind Speed             299
Current Description    299
dtype: int64

In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,72.23,overcast clouds,-23.1203,-134.9692,
2,Eyl,SO,83.23,broken clouds,7.9803,49.8164,
5,Victoria,HK,87.33,overcast clouds,22.2855,114.1577,
8,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
12,Tual,ID,81.55,scattered clouds,-5.6667,132.7500,
15,Benidorm,ES,75.49,overcast clouds,38.5382,-0.1310,
22,Bandarbeyla,SO,80.29,scattered clouds,9.4942,50.8122,
24,Kapaa,US,73.72,scattered clouds,22.0752,-159.3190,
26,Hellin,ES,74.98,overcast clouds,38.5106,-1.7010,
30,Palora,EC,71.71,overcast clouds,-1.7000,-77.9333,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found.. skipping)")
        

Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)
Hotel not found.. skipping)


In [35]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df.dropna(how="any")

In [37]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}<dd>and<dd>{Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
city_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [55]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = city_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))